In [59]:
import random
import math
import copy
from collections import deque


In [60]:
def read_top_instance(file_path):
    with open(file_path, 'r') as file:
        # Read the first three lines
        n_line = file.readline().strip()
        m_line = file.readline().strip()
        tmax_line = file.readline().strip()

        # Parse n N
        _, N = n_line.split()
        N = int(N)

        # Parse m P
        _, P = m_line.split()
        P = int(P)

        # Parse tmax Tmax
        _, Tmax = tmax_line.split()
        Tmax = float(Tmax)

        # Read the remaining lines for each node
        nodes = []
        for i in range(N):
            line = file.readline().strip()
            if line == '':
                continue  # Skip empty lines
            x_str, y_str, s_str = line.split()
            x = float(x_str)
            y = float(y_str)
            s = float(s_str)
            nodes.append({'id': i, 'x': x, 'y': y, 'score': s})

        return N, P, Tmax, nodes


In [61]:
# define problem instance. 
class ProblemInstance:
    def __init__(self, file_path):
        """
        Initialize the problem instance by reading data from a file.

        :param file_path: Path to the instance data file
        """
        self.N, self.P, self.Tmax, self.nodes = read_top_instance(file_path)
        self.distance_matrix = self.compute_distance_matrix()

    def compute_distance_matrix(self):
        """
        Compute the Euclidean distance matrix keyed by node IDs.

        :return: A dictionary of dictionaries representing the distance matrix, 
                where distance_matrix[id1][id2] = distance between node id1 and id2.
        """
        # Initialize the distance matrix as a nested dictionary
        distance_matrix = {}
        
        # Extract all node IDs and coordinates for convenience
        nodes_info = {node['id']: (node['x'], node['y']) for node in self.nodes}

        for id1, (x1, y1) in nodes_info.items():
            distance_matrix[id1] = {}
            for id2, (x2, y2) in nodes_info.items():
                if id1 == id2:
                    distance_matrix[id1][id2] = 0.0
                else:
                    distance = math.hypot(x1 - x2, y1 - y2)
                    distance_matrix[id1][id2] = distance

        return distance_matrix


In [62]:
class HALNS:
    def __init__(self, file_path, parameters):
        """
        Initialize the HALNS heuristic with the given problem instance file and parameters.

        :param file_path: Path to the TOP instance data file
        :param parameters: A dictionary containing algorithm parameters
        """
        self.problem = ProblemInstance(file_path)
        self.params = parameters

        # Initialize variables
        self.s_best = None
        self.s_adm = None
        self.T = self.params['T0']
        self.seg = 0
        self.iteration_best = 0

        # Initialize scores for strategies and operators
        # Mapped from the paper: dynamic profit/time, highest profit, random, LRFI
        self.selection_strategies = [
            'dynamic_profit_per_time', 
            'highest_profit',
            'random',
            'lrfi'
        ]

        self.removal_ops = [
            'random_removal',
            'lowest_profit_removal',
            'largest_saving_removal',
            'route_removal',
            'sequence_removal'
        ]
        
        self.insertion_ops = [
            'first_available',
            'last_available',
            'random_available',
            'best_overall_position',
            'best_position'
        ]

        self.selection_scores = {}
        self.selection_observed_scores = {}
        self.selection_counts = {}

        self.removal_scores = {}
        self.removal_observed_scores = {}
        self.removal_counts = {}

        self.insertion_scores = {}
        self.insertion_observed_scores = {}
        self.insertion_counts = {}

        # Keep track of last removed nodes for LRFI strategy
        self.last_removed_nodes = deque(maxlen=self.params.get('lrfi_count', 10))



    def _compute_route_time(self, route):
        dm = self.problem.distance_matrix
        time = 0.0
        for i in range(len(route)-1):
            time += dm[route[i]][route[i+1]]
        return time

    def _compute_solution_travel_time(self, solution):
        return sum(self._compute_route_time(r) for r in solution)

    def _is_feasible_insertion(self, route, pos, node):
        # Check feasibility of inserting node at route[pos]
        # For simplicity, assume feasibility = route time <= Tmax after insertion
        dm = self.problem.distance_matrix
        new_route = route[:pos] + [node] + route[pos:]
        return self._compute_route_time(new_route) <= self.problem.Tmax

    def _insert_node_in_position(self, route, pos, node):
        route.insert(pos, node)

    def _incremental_time_for_insertion(self, route, pos, node):
        dm = self.problem.distance_matrix
        before = dm[route[pos-1]][route[pos]]
        after = dm[route[pos-1]][node] + dm[node][route[pos]]
        return after - before

    def node_elimination_procedure(self):
        """
        Apply a node elimination procedure.
        This procedure removes nodes that cannot be serviced within the allowed time budget.
        A node i is eliminated if even the direct route s -> i -> e exceeds Tmax.
        """
        print("Applying node elimination procedure...")

        # Extract relevant information from the problem instance
        N = self.problem.N
        distance_matrix = self.problem.distance_matrix
        Tmax = self.problem.Tmax

        start_node = 0
        end_node = N - 1

        # Mandatory nodes (start and end) cannot be eliminated
        # We'll create a new list of nodes that remain feasible after elimination
        feasible_nodes = []

        # For each node, check feasibility:
        for node in self.problem.nodes:
            i = node['id']
            
            # Skip start and end nodes as they must remain
            if i == start_node or i == end_node:
                feasible_nodes.append(node)
                continue

            # Compute minimal route time s -> i -> e
            T_sie = distance_matrix[start_node][i] + distance_matrix[i][end_node]

            if T_sie <= Tmax:
                # This node could potentially be serviced in some feasible route
                feasible_nodes.append(node)
            else:
                # Node cannot be serviced within Tmax, so it is eliminated
                print(f"Eliminating node {i} due to infeasibility (s->i->e exceeds Tmax).")

        # Update the problem instance with the reduced set of nodes
        self.problem.nodes = feasible_nodes
        # Update N to reflect the new number of nodes
        self.problem.N = len(feasible_nodes)

        # Create a dictionary keyed by node id for direct access:
        self.problem.node_dict = {node['id']: node for node in self.problem.nodes}

        print("Node elimination procedure completed.")
    
    def construct_initial_solution(self):
        print("Constructing initial solution using the nearest neighbor algorithm...")

        N = self.problem.N
        start_node = 0
        end_node = N - 1

        unvisited = {nid for nid in self.problem.node_dict.keys() if nid not in {start_node, end_node}}
        paths = [[start_node, end_node] for _ in range(self.problem.P)]
        current_times = []
        for p in range(self.problem.P):
            initial_time = self.problem.distance_matrix[start_node][end_node]
            current_times.append(initial_time)

        current_positions = [start_node for _ in range(self.problem.P)]

        while unvisited:
            improvement = False
            for p in range(self.problem.P):
                last_node = current_positions[p]
                best_node = None
                best_score = -1
                best_distance = float('inf')

                for node_id in unvisited:
                    distance = self.problem.distance_matrix[last_node][node_id]
                    distance_to_end = self.problem.distance_matrix[node_id][end_node]
                    new_total_time = current_times[p] - self.problem.distance_matrix[last_node][end_node] + distance + distance_to_end

                    if new_total_time <= self.problem.Tmax:
                        node_score = self.problem.node_dict[node_id]['score']  
                        heuristic = node_score / (distance + 1)
                        if heuristic > best_score:
                            best_score = heuristic
                            best_node = node_id
                            best_distance = distance

                if best_node is not None:
                    paths[p].insert(-1, best_node)
                    unvisited.remove(best_node)
                    current_times[p] = current_times[p] - self.problem.distance_matrix[last_node][end_node] + best_distance + self.problem.distance_matrix[best_node][end_node]
                    current_positions[p] = best_node
                    improvement = True
                    print(f"Path {p + 1}: Added node {best_node} (Score: {self.problem.node_dict[best_node]['score']}, Distance: {best_distance:.2f})")

            if not improvement:
                print("No further nodes can be added without exceeding Tmax.")
                break

        for p in range(self.problem.P):
            if paths[p][-1] != end_node:
                paths[p].append(end_node)
                print(f"Path {p + 1}: Ensured ending at node {end_node}. Total time: {current_times[p]:.2f}")

        self.s_best = paths
        self.s_adm = copy.deepcopy(paths)
        print("Initial solution constructed with time budget consideration.")
        return paths
        
    def initialize_scores(self):
        """
        Initialize the scores of the node selection strategy and the removal and insertion operators.
        """
        print("Initializing scores for strategies and operators...")

        # Each strategy/operator starts with a score of 1.0 as per the described approach
        for strat in self.selection_strategies:
            self.selection_scores[strat] = 1.0
            self.selection_observed_scores[strat] = 0.0
            self.selection_counts[strat] = 0

        for rem in self.removal_ops:
            self.removal_scores[rem] = 1.0
            self.removal_observed_scores[rem] = 0.0
            self.removal_counts[rem] = 0

        for ins in self.insertion_ops:
            self.insertion_scores[ins] = 1.0
            self.insertion_observed_scores[ins] = 0.0
            self.insertion_counts[ins] = 0
    
    def select_node_selection_strategy(self):
        """
        Select a node selection strategy based on current scores.
        A roulette wheel selection is performed:
        P_k = p_k / sum_of_all_p
        The strategies are:
        1. dynamic_profit_per_time
        2. highest_profit
        3. random
        4. lrfi
        """
        print("Selecting node selection strategy...")
        total_score = sum(self.selection_scores.values())
        r = random.uniform(0, total_score)
        cumulative = 0.0
        for strat, score in self.selection_scores.items():
            cumulative += score
            if r <= cumulative:
                return strat
        # Fallback in case of numerical issues
        return self.selection_strategies[0]
    
    def select_removal_insertion_operators(self):
        """
        Select removal and insertion operators based on current scores via roulette wheel selection.
        """
        print("Selecting removal and insertion operators...")

        # Removal operator selection
        total_score_rem = sum(self.removal_scores.values())
        r_rem = random.uniform(0, total_score_rem)
        cumulative_rem = 0.0
        selected_removal = None
        for rem, score in self.removal_scores.items():
            cumulative_rem += score
            if r_rem <= cumulative_rem:
                selected_removal = rem
                break

        # Insertion operator selection
        total_score_ins = sum(self.insertion_scores.values())
        r_ins = random.uniform(0, total_score_ins)
        cumulative_ins = 0.0
        selected_insertion = None
        for ins, score in self.insertion_scores.items():
            cumulative_ins += score
            if r_ins <= cumulative_ins:
                selected_insertion = ins
                break

        return selected_removal, selected_insertion
    
    def remove_nodes(self, solution, b, removal_operator):
            print(f"Removing {b} nodes using {removal_operator}...")
            
            if removal_operator == 'random_removal':
                removed = self._remove_random_nodes(solution, b)
            elif removal_operator == 'lowest_profit_removal':
                removed = self._remove_lowest_profit_nodes(solution, b)
            elif removal_operator == 'largest_saving_removal':
                removed = self._remove_largest_saving_nodes(solution, b)
            elif removal_operator == 'route_removal':
                removed = self._remove_route(solution)
            elif removal_operator == 'sequence_removal':
                removed = self._remove_sequence(solution, b)
            else:
                # Default to random removal if unknown
                removed = self._remove_random_nodes(solution, b)
            
            # Update solution by removing chosen nodes
            for route in solution:
                route[:] = [x for x in route if x not in removed]

            # Update LRFI queue
            for n in removed:
                self.last_removed_nodes.appendleft(n)
            
            return solution

    def _remove_random_nodes(self, solution, b):
        nodes = [n for route in solution for n in route if n not in (0, self.problem.N - 1)]
        removed = set()
        for _ in range(min(b, len(nodes))):
            n = random.choice(nodes)
            nodes.remove(n)
            removed.add(n)
        return removed

    def _remove_lowest_profit_nodes(self, solution, b):
        # Weighted removal based on lowest profit: nodes with lower profit are more likely to be chosen
        nodes = [n for route in solution for n in route if n not in (0, self.problem.N - 1)]
        if not nodes:
            return set()
        
        # Sort by profit ascending
        nodes.sort(key=lambda x: self.problem.node_dict[x]['score'])
        total_profit = sum(self.problem.node_dict[n]['score'] for n in nodes) + 1e-9
        removed = set()
        for _ in range(min(b, len(nodes))):
            # Roulette wheel giving higher chance to lower profit nodes
            r = random.uniform(0, total_profit)
            cumulative = 0.0
            chosen = nodes[0]
            for n in nodes:
                cumulative += self.problem.node_dict[n]['score']
                if r <= cumulative:
                    chosen = n
                    break
            removed.add(chosen)
            nodes.remove(chosen)
            total_profit -= self.problem.node_dict[chosen]['score']
        return removed

    def _remove_largest_saving_nodes(self, solution, b):
        # Compute saving in travel time for removing each node: D(s) - D(s\i)
        # First compute D(s)
        original_time = self._compute_solution_travel_time(solution)

        # For each node, compute saving
        candidates = []
        for r_idx, route in enumerate(solution):
            for i in range(1, len(route)-1):
                n = route[i]
                # Temporarily remove node n and compute new time
                temp_route = route[:i] + route[i+1:]
                temp_sol = solution[:r_idx] + [temp_route] + solution[r_idx+1:]
                new_time = self._compute_solution_travel_time(temp_sol)
                saving = original_time - new_time
                candidates.append((n, saving))
        
        if not candidates:
            return set()

        # Sort by saving descending, then use roulette wheel with weights = saving
        total_saving = sum(s for _, s in candidates) + 1e-9
        removed = set()
        nodes_list = candidates[:]
        # Remove b nodes with preference to largest saving
        for _ in range(min(b, len(nodes_list))):
            r = random.uniform(0, total_saving)
            cumulative = 0.0
            chosen = nodes_list[0][0]
            for (nd, sv) in nodes_list:
                cumulative += sv
                if r <= cumulative:
                    chosen = nd
                    break
            removed.add(chosen)
            # Remove chosen from candidate list and adjust total_saving
            for idx, (nd, sv) in enumerate(nodes_list):
                if nd == chosen:
                    total_saving -= sv
                    del nodes_list[idx]
                    break
        return removed

    def _remove_route(self, solution):
        # Randomly select a route (other than maybe a trivial one) and remove all nodes from it
        non_empty_routes = [r for r in solution if len(r) > 2]
        if not non_empty_routes:
            # If no non-empty routes, nothing to remove
            return set()
        route = random.choice(non_empty_routes)
        # Remove all non-depot nodes
        removed = set(route[1:-1])
        return removed

    def _remove_sequence(self, solution, b):
        # Randomly select a route and remove a sequence of b consecutive nodes (excluding depots)
        routes_with_enough_nodes = [r for r in solution if len(r)-2 >= b]
        if not routes_with_enough_nodes:
            # Fall back to random removal if not possible
            return self._remove_random_nodes(solution, b)
        route = random.choice(routes_with_enough_nodes)
        start_idx = random.randint(1, len(route)-1 - b)
        removed = set(route[start_idx:start_idx+b])
        return removed

    # insertion operators
    def insert_nodes(self, solution, insertion_operator, strategy):
        print(f"Inserting nodes using {insertion_operator} with strategy {strategy}...")
        
        # Determine nodes to insert based on strategy (already chosen and ordered)
        visited = {nid for r in solution for nid in r}
        start_node = 0
        end_node = self.problem.N - 1
        unvisited = set(self.problem.node_dict.keys()) - visited - {start_node, end_node}

        # Strategy decides the order in which nodes are considered. This is handled separately.
        # Here we assume the `strategy` has already selected insertion_order in a previous step.
        # For demonstration, let's assume we have the insertion_order from node selection strategy step:
        # In real code, you'd integrate your node selection strategy logic here.
        insertion_order = self._get_insertion_order(unvisited, strategy)

        # Apply the chosen insertion operator to insert nodes in the given order
        if insertion_operator == 'first_available':
            solution = self._insert_nodes_first_available(solution, insertion_order)
        elif insertion_operator == 'last_available':
            solution = self._insert_nodes_last_available(solution, insertion_order)
        elif insertion_operator == 'random_available':
            solution = self._insert_nodes_random_available(solution, insertion_order)
        elif insertion_operator == 'best_overall_position':
            solution = self._insert_nodes_best_overall_position(solution, insertion_order)
        elif insertion_operator == 'best_position':
            solution = self._insert_nodes_best_position(solution, insertion_order)
        else:
            # Default to a simple operator, e.g., first_available
            solution = self._insert_nodes_first_available(solution, insertion_order)
        
        return solution

    def _get_insertion_order(self, unvisited, strategy):
        """
        Determine the order in which nodes from 'unvisited' should be considered for insertion
        based on the selected node selection strategy.
        Strategies:
        - dynamic_profit_per_time
        - highest_profit
        - random
        - lrfi
        """
        if not unvisited:
            return []

        if strategy == 'dynamic_profit_per_time':
            # Compute p_i / D(s+i) for each node i
            candidates = []
            for n in unvisited:
                profit = self.problem.node_dict[n]['score']
                incremental_time = self._compute_min_incremental_time(self.s_adm, n)
                ratio = profit / incremental_time if incremental_time > 0 else float('inf')
                candidates.append((n, ratio))
            # Sort by ratio descending
            candidates.sort(key=lambda x: x[1], reverse=True)
            insertion_list = [n for n, _ in candidates]

        elif strategy == 'highest_profit':
            # Roulette wheel based on profit
            temp_nodes = list(unvisited)
            insertion_list = []
            total_profit = sum(self.problem.node_dict[n]['score'] for n in temp_nodes) + 1e-9
            while temp_nodes:
                r = random.uniform(0, total_profit)
                cumulative = 0.0
                chosen = temp_nodes[0]
                for node_id in temp_nodes:
                    cumulative += self.problem.node_dict[node_id]['score']
                    if r <= cumulative:
                        chosen = node_id
                        break
                insertion_list.append(chosen)
                temp_nodes.remove(chosen)
                total_profit -= self.problem.node_dict[chosen]['score']

        elif strategy == 'random':
            insertion_list = list(unvisited)
            random.shuffle(insertion_list)

        elif strategy == 'lrfi':
            # Insert last removed nodes first if they are still unvisited
            lrfi_candidates = [n for n in self.last_removed_nodes if n in unvisited]
            # Remove them from unvisited
            remainder = list(unvisited - set(lrfi_candidates))
            # For remainder, fallback to random (or another strategy if desired)
            random.shuffle(remainder)
            insertion_list = lrfi_candidates + remainder

        else:
            # Default fallback - random
            insertion_list = list(unvisited)
            random.shuffle(insertion_list)

        return insertion_list


    def _compute_min_incremental_time(self, solution, node):
        """
        Compute the minimal incremental travel time of inserting 'node' anywhere in the given 'solution'.
        """
        min_increment = float('inf')
        dm = self.problem.distance_matrix
        for r_idx, route in enumerate(solution):
            for i in range(len(route)-1):
                a, b = route[i], route[i+1]
                current_time = dm[a][b]
                new_time = dm[a][node] + dm[node][b]
                increment = new_time - current_time
                if increment < min_increment:
                    min_increment = increment
        return min_increment if min_increment != float('inf') else 1e9  # Large number if no feasible insertion

    # Insertion methods
    def _insert_nodes_first_available(self, solution, nodes):
        # For each node, insert at the first feasible position in some route
        for node in nodes:
            # Try each route in order, from start to end
            inserted = False
            for route in solution:
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        self._insert_node_in_position(route, pos, node)
                        inserted = True
                        break
                if inserted:
                    break
        return solution

    def _insert_nodes_last_available(self, solution, nodes):
        # For each node, insert at the last feasible position scanning routes from end to start
        for node in nodes:
            inserted = False
            for route in solution:
                for pos in range(len(route)-1, 0, -1):
                    if self._is_feasible_insertion(route, pos, node):
                        self._insert_node_in_position(route, pos, node)
                        inserted = True
                        break
                if inserted:
                    break
        return solution

    def _insert_nodes_random_available(self, solution, nodes):
        # For each node, find all feasible positions and choose one at random
        for node in nodes:
            candidates = []
            for r_idx, route in enumerate(solution):
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        candidates.append((r_idx, pos))
            if candidates:
                r_idx, pos = random.choice(candidates)
                self._insert_node_in_position(solution[r_idx], pos, node)
        return solution

    def _insert_nodes_best_overall_position(self, solution, nodes):
        # For each node, test all feasible insertions and choose the one minimizing total solution travel time
        # This can be expensive, but we do it as described.
        original_time = self._compute_solution_travel_time(solution)
        for node in nodes:
            best_delta = float('inf')
            best_route = None
            best_pos = None
            for r_idx, route in enumerate(solution):
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        # Insert temporarily and check total time
                        temp_sol = copy.deepcopy(solution)
                        self._insert_node_in_position(temp_sol[r_idx], pos, node)
                        new_time = self._compute_solution_travel_time(temp_sol)
                        delta = new_time - original_time
                        if delta < best_delta:
                            best_delta = delta
                            best_route = r_idx
                            best_pos = pos
            if best_route is not None:
                self._insert_node_in_position(solution[best_route], best_pos, node)
                # Update original_time after insertion
                original_time += best_delta
        return solution

    def _insert_nodes_best_position(self, solution, nodes):
        # For each node, find the insertion that minimizes local incremental time
        for node in nodes:
            best_increment = float('inf')
            best_route = None
            best_pos = None
            for r_idx, route in enumerate(solution):
                for pos in range(1, len(route)):
                    if self._is_feasible_insertion(route, pos, node):
                        increment = self._incremental_time_for_insertion(route, pos, node)
                        if increment < best_increment:
                            best_increment = increment
                            best_route = r_idx
                            best_pos = pos
            if best_route is not None:
                self._insert_node_in_position(solution[best_route], best_pos, node)
        return solution
    def evaluate_solution(self, solution):
        visited = set()
        for route in solution:
            for nid in route:
                visited.add(nid)
                
        total_score = 0.0
        for nid in visited:
            total_score += self.problem.node_dict[nid]['score']
        
        return total_score
    
    def apply_local_search(self, solution):
        """
        Apply local search procedures on the solution.
        Placeholder for the actual implementation.
        
        :param solution: Solution to improve
        :return: Improved solution
        """
        print("Applying local search procedures...")
        # TODO: Implement local search
        improved_solution = copy.deepcopy(solution)  # Replace with actual improvement
        return improved_solution
    
    def generate_and_solve_SROP(self, solution):
        """
        Generate and solve a SROP (Sub-Route Optimization Problem).
        Placeholder for the actual implementation.
        
        :param solution: Current solution
        :return: Optimized solution
        """
        print("Generating and solving SROP...")
        # TODO: Implement SROP generation and solving
        optimized_solution = copy.deepcopy(solution)  # Replace with actual optimization
        return optimized_solution
    
    def generate_and_solve_SPP(self, solution):
        """
        Generate and solve a SPP (Single Path Problem).
        Placeholder for the actual implementation.
        
        :param solution: Current solution
        :return: Optimized solution
        """
        print("Generating and solving SPP...")
        # TODO: Implement SPP generation and solving
        optimized_solution = copy.deepcopy(solution)  # Replace with actual optimization
        return optimized_solution
    
    def update_scores(self):
        """
        Update the scores of the ALNS operators and insertion strategies at the end of a segment.
        
        p_{k,q+1} = kappa * (observed_score_k / n_k) + (1 - kappa)*p_{k,q}, if n_k > 0
        If n_k = 0, score remains unchanged.
        After updating, observed scores and counts reset.
        """
        print("Updating scores of the ALNS operators and insertion strategies...")

        kappa = self.params.get('kappa', 0.5)

        def update_category_scores(scores, observed_scores, counts):
            for k in scores.keys():
                n_k = counts[k]
                if n_k > 0:
                    old_score = scores[k]
                    new_score = kappa * (observed_scores[k] / n_k) + (1 - kappa) * old_score
                    scores[k] = new_score
                # Reset for next segment
                observed_scores[k] = 0.0
                counts[k] = 0

        update_category_scores(self.selection_scores, self.selection_observed_scores, self.selection_counts)
        update_category_scores(self.removal_scores, self.removal_observed_scores, self.removal_counts)
        update_category_scores(self.insertion_scores, self.insertion_observed_scores, self.insertion_counts)

        print("Score update completed.")
        
    def run(self):
            # Step 1: Apply node elimination procedure
            self.node_elimination_procedure()
            
            # Step 2: Construct initial solution using the nearest neighbor algorithm
            initial_solution = self.construct_initial_solution()
            self.s_best = initial_solution
            self.s_adm = initial_solution
            
            # Initialize parameters
            self.T = self.params['T0']
            self.seg = 0
            self.iteration_best = 0
            
            # Step 4: Initialize scores
            self.initialize_scores()

            # Extract q parameters
            q1 = self.params.get('q1', 10.0)
            q2 = self.params.get('q2', 5.0)
            q3 = self.params.get('q3', 1.0)

            # Step 5: Main loop
            while self.seg < self.params['Nseg'] and self.iteration_best < self.params['iteration_best_max']:
                print(f"\n--- Segment {self.seg + 1} ---")
                iteration = 0
                
                while iteration < self.params['iteration_max']:
                    print(f"\nIteration {iteration + 1} within Segment {self.seg + 1}")
                    s = copy.deepcopy(self.s_adm)
                    
                    # Generate b (number of nodes to remove)
                    # For safety, ensure b does not exceed solution complexity; using a simple logic:
                    b = random.randint(1, max(1, int(0.25*sum(len(p) for p in s)-2*len(s))))
                    print(f"Number of nodes to remove: {b}")
                    
                    # Select node selection strategy
                    c = self.select_node_selection_strategy()
                    
                    # Select removal and insertion operators
                    R, I = self.select_removal_insertion_operators()
                    
                    # Count usage
                    self.selection_counts[c] += 1
                    self.removal_counts[R] += 1
                    self.insertion_counts[I] += 1

                    # Remove b nodes using R
                    s = self.remove_nodes(s, b, R)
                    
                    # Insert nodes using I following c
                    s = self.insert_nodes(s, I, c)
                    
                    # Generate a random number d in [0,1]
                    d = random.uniform(0, 1)
                    print(f"Random number d: {d}")
                    
                    # Evaluate solutions
                    f_s = self.evaluate_solution(s)
                    f_sadm = self.evaluate_solution(self.s_adm)
                    f_sbest = self.evaluate_solution(self.s_best)
                    
                    if f_s >= f_sadm or d <= math.exp((f_s - f_sadm) / self.T):
                        print("Solution accepted based on acceptance criteria.")
                        
                        # Determine q increment based on improvement
                        if f_s > f_sbest:
                            q_value = q1
                        elif f_s > f_sadm:
                            q_value = q2
                        else:
                            q_value = q3

                        # Increment observed scores
                        self.selection_observed_scores[c] += q_value
                        self.removal_observed_scores[R] += q_value
                        self.insertion_observed_scores[I] += q_value

                        # If f(s) > f(sadm), apply local search
                        if f_s > f_sadm:
                            s = self.apply_local_search(s)
                            f_s = self.evaluate_solution(s)
                            print("Applied local search.")

                        # If f(s) > f(s_best), update sbest and reset iteration_best
                        if f_s > f_sbest:
                            print("New best solution found. Generating and solving SROP...")
                            s = self.generate_and_solve_SROP(s)
                            self.s_best = copy.deepcopy(s)
                            self.iteration_best = 0
                            print("s_best updated.")
                        else:
                            self.iteration_best += 1
                            print(f"Iteration best incremented to {self.iteration_best}.")
                        
                        self.s_adm = copy.deepcopy(s)
                    else:
                        # Not accepted
                        self.iteration_best += 1
                        print(f"Solution not accepted. Iteration best incremented to {self.iteration_best}.")
                    
                    # Temperature check
                    if self.T <= self.params['T_min']:
                        print("Temperature below minimum. Resetting temperature and generating SPP...")
                        self.T = self.params['T0']
                        s = self.generate_and_solve_SPP(s)
                        
                        if self.evaluate_solution(s) > self.evaluate_solution(self.s_best):
                            self.iteration_best = 0
                            print("Improved solution found after SPP.")
                        
                        self.s_best = copy.deepcopy(s)
                        self.s_adm = copy.deepcopy(s)
                    
                    # Update temperature and iteration
                    self.T *= self.params['cooling_rate']
                    iteration += 1
                    print(f"Temperature updated to {self.T}.")
                
                # Update scores at the end of the segment
                self.update_scores()
                
                # Increment segment
                self.seg += 1
                print(f"Segment {self.seg} completed.")
            
            # Generate and solve a final SPP
            print("\nGenerating and solving final SPP...")
            self.s_best = self.generate_and_solve_SPP(self.s_best)
            
            # Final solution
            print("Final solution obtained.")
            return self.s_best


In [74]:
if __name__ == "__main__":
    # Define the path to your TOP instance data file
    file_path = 'Set_100_234/p4.2.a.txt' # 3 paths does not properly work currently!

    parameters = {
        'T0': 95,              # Initial temperature for the simulated annealing mechanism.
        'T_min': 0.0001,              # Minimum temperature below which we reset temperature and solve a SPP.
        'cooling_rate': 0.9999,    # The factor by which the temperature is multiplied after each iteration.
        'Nseg': 100,              # Number of segments (run segments) after which score updates occur.
        'iteration_max': 1000,  # Maximum number of iterations per segment.
        'iteration_best_max': 50,# Maximum number of consecutive iterations without improvement before stopping.
        'kappa': 0.85,            # Reaction factor for updating operator/strategy scores.
        'q1': 20.0,              # Score increment if a new best solution is found.
        'q2': 5.0,               # Score increment if the solution improves the last admissible one but is not a best.
        'q3': 1.0,               # Score increment if the solution is accepted but does not improve the last admissible.
        'lrfi_count': 10         # Maximum number of nodes to track for "Last Removed First Inserted" strategy.
    }

    # Initialize HALNS heuristic with the problem instance file and parameters
    halns = HALNS(file_path, parameters)

    # Run HALNS to obtain the best solution
    best_solution = halns.run()

    # Output the best solution
    print("Best Solution:", best_solution)

    total_prize = halns.evaluate_solution(best_solution)
    print("Total Prize Collected:", total_prize)

Applying node elimination procedure...
Eliminating node 1 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 2 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 3 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 5 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 8 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 11 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 12 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 13 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 15 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 16 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 17 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 18 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 19 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 20 due to infeasibility (s->i->e exceeds Tmax).
Eliminating node 21 due to infeasibility (s->i->e exceed

In [71]:
n,p,t,nodes = read_top_instance(file_path)

In [72]:
n,p,t,nodes 

(100,
 2,
 25.0,
 [{'id': 0, 'x': 18.19, 'y': 6.32, 'score': 0.0},
  {'id': 1, 'x': 15.52, 'y': 28.03, 'score': 7.0},
  {'id': 2, 'x': 9.0, 'y': 28.01, 'score': 5.0},
  {'id': 3, 'x': 16.93, 'y': 2.09, 'score': 24.0},
  {'id': 4, 'x': 8.64, 'y': 19.85, 'score': 3.0},
  {'id': 5, 'x': 18.43, 'y': 29.93, 'score': 23.0},
  {'id': 6, 'x': 3.94, 'y': 10.77, 'score': 3.0},
  {'id': 7, 'x': 14.78, 'y': 7.61, 'score': 26.0},
  {'id': 8, 'x': 2.41, 'y': 25.06, 'score': 24.0},
  {'id': 9, 'x': 18.41, 'y': 11.88, 'score': 16.0},
  {'id': 10, 'x': 14.91, 'y': 17.24, 'score': 21.0},
  {'id': 11, 'x': 7.14, 'y': 0.48, 'score': 6.0},
  {'id': 12, 'x': 13.44, 'y': 24.76, 'score': 17.0},
  {'id': 13, 'x': 17.54, 'y': 29.43, 'score': 27.0},
  {'id': 14, 'x': 16.71, 'y': 9.5, 'score': 27.0},
  {'id': 15, 'x': 12.38, 'y': 26.66, 'score': 1.0},
  {'id': 16, 'x': 25.69, 'y': 6.34, 'score': 6.0},
  {'id': 17, 'x': 11.62, 'y': 24.0, 'score': 27.0},
  {'id': 18, 'x': 10.24, 'y': 3.02, 'score': 23.0},
  {'id': 

In [76]:
halns.problem.node_dict

{0: {'id': 0, 'x': 18.19, 'y': 6.32, 'score': 0.0},
 4: {'id': 4, 'x': 8.64, 'y': 19.85, 'score': 3.0},
 6: {'id': 6, 'x': 3.94, 'y': 10.77, 'score': 3.0},
 7: {'id': 7, 'x': 14.78, 'y': 7.61, 'score': 26.0},
 9: {'id': 9, 'x': 18.41, 'y': 11.88, 'score': 16.0},
 10: {'id': 10, 'x': 14.91, 'y': 17.24, 'score': 21.0},
 14: {'id': 14, 'x': 16.71, 'y': 9.5, 'score': 27.0},
 23: {'id': 23, 'x': 15.88, 'y': 5.54, 'score': 21.0},
 24: {'id': 24, 'x': 1.32, 'y': 19.32, 'score': 23.0},
 30: {'id': 30, 'x': 20.9, 'y': 7.67, 'score': 8.0},
 34: {'id': 34, 'x': 13.57, 'y': 9.41, 'score': 11.0},
 39: {'id': 39, 'x': 7.0, 'y': 21.44, 'score': 8.0},
 43: {'id': 43, 'x': 13.51, 'y': 8.05, 'score': 1.0},
 47: {'id': 47, 'x': 16.32, 'y': 3.73, 'score': 3.0},
 49: {'id': 49, 'x': 1.04, 'y': 13.47, 'score': 6.0},
 52: {'id': 52, 'x': 11.96, 'y': 15.15, 'score': 17.0},
 55: {'id': 55, 'x': 6.52, 'y': 19.42, 'score': 17.0},
 60: {'id': 60, 'x': 4.46, 'y': 7.85, 'score': 7.0},
 62: {'id': 62, 'x': 6.82, 'y'

In [75]:
halns.problem.N,halns.problem.P 

(35, 2)

In [64]:
visited = set()
for route in best_solution:
    for nid in route:
        visited.add(nid)

print("Visited Nodes:", visited)

total_score = 0.0
for nid in visited:
    score = halns.problem.node_dict[nid]['score'] if nid in halns.problem.node_dict else None
    print(f"Node {nid}: Score={score}")
    if score is not None:
        total_score += score
print("Total Prize Collected:", total_score)


Visited Nodes: {0, 96, 34, 67, 69, 7, 10, 43, 14, 80, 52, 85, 84, 23, 87, 94}
Node 0: Score=0.0
Node 96: Score=24.0
Node 34: Score=11.0
Node 67: Score=17.0
Node 69: Score=4.0
Node 7: Score=26.0
Node 10: Score=21.0
Node 43: Score=1.0
Node 14: Score=27.0
Node 80: Score=19.0
Node 52: Score=17.0
Node 85: Score=1.0
Node 84: Score=24.0
Node 23: Score=21.0
Node 87: Score=11.0
Node 94: Score=8.0
Total Prize Collected: 232.0
